In [1]:
cd ..

/Users/muhammadfarooq/Codebase/wood-log-detection/Pith2Bark


In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scikitplot as skplt

import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, Dropout

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from src.cv_winner import winner_model_rings
from src.damage import merge_df
from src.damage import read_damage_min_max
from src.damage import read_damage_mean

# Damage Dataset

In [3]:
rmse_url = "cv_output/ringsranking_rmse.csv"
rings_url = "cv_output/ringsranking_rings.csv"
df_ringsranking = winner_model_rings(rmse_url, rings_url)

Polar Pith Canny(blur otsu) 12.829 (subseq)


In [4]:
rmse_url = "cv_output/pine_rmse.csv"
rings_url = "cv_output/pine_rings.csv"
df_pine = winner_model_rings(rmse_url, rings_url)

Polar Pith Canny(blur triangle) 19.888 (peaks)


In [5]:
rmse_url = "cv_output/fur_rmse.csv"
rings_url = "cv_output/fur_rings.csv"
df_fur = winner_model_rings(rmse_url, rings_url)

Polar Pith Canny(blur triangle) 13.429 (peaks)


In [6]:
rmse_url = "cv_output/tracy_rmse.csv"
rings_url = "cv_output/tracy_rings.csv"
df_tracy = winner_model_rings(rmse_url, rings_url)

Polar Canny(triangle) 39.684 (peaks)


In [7]:
rmse_url = "cv_output/data_rmse.csv"
rings_url = "cv_output/data_rings.csv"
df_data = winner_model_rings(rmse_url, rings_url)

Canny(triangle) 18.79 (subseq)


In [8]:
#damage
damage_url = "damage_dataset/damage.csv"

#merge
df_merge = merge_df(df_ringsranking, df_pine, df_fur, df_tracy, df_data)

df_damage_min_max = read_damage_min_max(damage_url, df_merge)

df_damage_mean = read_damage_mean(damage_url, df_merge)

In [9]:
#STOP

# Config

In [10]:
training_percent = 0.9

epochs, batch_size = 50, 32
kf = KFold(n_splits=3, shuffle=True, random_state=42)

# mode = max for monitro val_accuracy and val_prc
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=10,
    restore_best_weights=True)

METRICS = [
    tf.keras.metrics.TruePositives(name='tp'),
    tf.keras.metrics.FalsePositives(name='fp'),
    tf.keras.metrics.TrueNegatives(name='tn'),
    tf.keras.metrics.FalseNegatives(name='fn'), 
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='auc'),
    tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

#columns to drop
columns = ['image', 'algo']

In [11]:
def heat_map_normalized(y_true, y_pred):
    
    class_names = ['Not Damage', 'Damaged']
    
    skplt.metrics.plot_confusion_matrix(y_true, y_pred,
                                        figsize=(4,3),
                                        normalize=True)
    
    # Customize axis tick labels
    plt.xticks([0,1], class_names)
    plt.yticks([0,1], class_names)
    
    plt.yticks(rotation=90)
    
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.show()

In [12]:
def heat_map(_cm):
    # plot confusion matrix as heatmap
    labels = ['Not Damage', 'Damaged']
    
    # Set up the matplotlib figure
    plt.figure(figsize=(4, 3))
    
    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(230, 20, as_cmap=True)
    
    sns.heatmap(_cm, annot=True, cmap=cmap, xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.show()

In [13]:
units = [256]
activations = ['relu', 'tanh', 'sigmoid']
kernel_size = [3, 5]
learning_rate = [0.001, 0.0001, 0.00001]
optimizer = ['Adam', 'RMSprop', 'Nadam']
dropout = [0.1, 0.2, 0.3]

# Model

In [14]:
class Baseline():
    def __init__(self, _input_dim, unit, activation, kernel_size, learning_rate, optimizer, dropout, metrics=METRICS):
        # Define the model architecture
        model = Sequential()
        
        # hidden layer
        model.add(Dense(unit, activation=activation, input_dim=_input_dim))
        model.add(Dropout(dropout))
        
        # hidden layer
        model.add(Dense(unit, activation=activation))
        model.add(Dropout(dropout))
        
        # hidden layer
        model.add(Dense(unit, activation=activation))
        model.add(Dropout(dropout))
        
        # hidden layer
        model.add(Dense(unit, activation=activation))
        model.add(Dropout(dropout))
        
        # output layer
        model.add(Dense(1, activation='sigmoid'))

        if optimizer == 'Adam':
            ops = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        elif optimizer == 'RMSprop':
            ops = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
        elif optimizer == 'Nadam':
            ops = tf.keras.optimizers.Nadam(learning_rate=learning_rate)
            
        # Compile the model
        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        self.model = model
        
    def train(self, X_train, y_train, X_val, y_val, epochs, batch_size):
        history = self.model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size, callbacks=[early_stopping], verbose=2)
        return history

    def evaluate(self, X_test, y_test):
        results = self.model.evaluate(X_test, y_test, verbose=0)
        return results

    def predict(self, X): #X_train X_test
        predictions = self.model.predict(X)
        return predictions
    
    def summary(self):
        self.model.summary()
        
    def metrics_names(self):
        return self.model.metrics_names

# Model MinMax

In [15]:
df = df_damage_min_max.drop(columns=columns)

X = df.drop(columns=['Damage'])
y = df['Damage']

# split the data into a holdout set and the rest for K-Fold cross-validation
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, train_size=training_percent, random_state=42)

In [16]:
# Initialize StandardScaler
scaler = StandardScaler()

# Fit scaler on training data
scaler.fit(X_train)

# Transform both training and test data
X_train_scaled = scaler.transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

In [17]:
best_acc = 0.0
baseline_values = { 'units': 0, 'activation': '', 'kernel_size': 0, 'learning_rate' : 0.0, 'optimizer': '', 'dropout': 0.0 }

In [18]:
input_dim = X_train.shape[1]

In [19]:
for u in units:
    for a in activations:
        for k in kernel_size:
            for l in learning_rate:
                for o in optimizer:
                    for d in dropout:
                        print('\nUnits:', u, 'Activation:', a, 'Kernel size:', k, 'Learning rate:', l, 'Optimizer:', o, 'Dropout:', d)
                        for train_index, val_index in kf.split(X_train_scaled):
                            # split the dataset into training and validation sets for this fold
                            X_train_kf, X_val_kf = X_train_scaled[train_index], X_train_scaled[val_index]
                            y_train_kf, y_val_kf = y_train.iloc[train_index], y_train.iloc[val_index]

                            model = Baseline(input_dim, u, a, k, l, o, d)

                            history = model.train(X_train_kf, y_train_kf, X_val_kf, y_val_kf, epochs, batch_size)
                            if (history.history['val_accuracy'][-1] > best_acc):
                                best_acc = history.history['val_accuracy'][-1]
                                baseline_values = { 'units': u, 'activation': a, 'kernel_size': k, 'learning_rate' : l, 'optimizer': o, 'dropout': d }


Units: 256 Activation: relu Kernel size: 3 Learning rate: 0.001 Optimizer: Adam Dropout: 0.1
Epoch 1/50
3/3 - 1s - loss: 0.6224 - accuracy: 0.6705 - val_loss: 0.4529 - val_accuracy: 0.9111 - 1s/epoch - 490ms/step
Epoch 2/50
3/3 - 0s - loss: 0.3495 - accuracy: 0.9091 - val_loss: 0.3147 - val_accuracy: 0.9111 - 40ms/epoch - 13ms/step
Epoch 3/50
3/3 - 0s - loss: 0.2407 - accuracy: 0.8864 - val_loss: 0.3200 - val_accuracy: 0.9111 - 36ms/epoch - 12ms/step
Epoch 4/50
3/3 - 0s - loss: 0.2222 - accuracy: 0.8864 - val_loss: 0.3780 - val_accuracy: 0.9111 - 35ms/epoch - 12ms/step
Epoch 5/50
3/3 - 0s - loss: 0.2240 - accuracy: 0.9091 - val_loss: 0.4232 - val_accuracy: 0.9111 - 36ms/epoch - 12ms/step
Epoch 6/50
3/3 - 0s - loss: 0.2090 - accuracy: 0.9205 - val_loss: 0.4359 - val_accuracy: 0.9111 - 35ms/epoch - 12ms/step
Epoch 7/50
3/3 - 0s - loss: 0.1497 - accuracy: 0.9659 - val_loss: 0.3878 - val_accuracy: 0.9111 - 36ms/epoch - 12ms/step
Epoch 8/50
3/3 - 0s - loss: 0.1257 - accuracy: 0.9773 - val_

Epoch 8/50
3/3 - 0s - loss: 0.1795 - accuracy: 0.9659 - val_loss: 0.4237 - val_accuracy: 0.9111 - 36ms/epoch - 12ms/step
Epoch 9/50
3/3 - 0s - loss: 0.1360 - accuracy: 0.9886 - val_loss: 0.4150 - val_accuracy: 0.9111 - 35ms/epoch - 12ms/step
Epoch 10/50
3/3 - 0s - loss: 0.0947 - accuracy: 0.9773 - val_loss: 0.4133 - val_accuracy: 0.9111 - 47ms/epoch - 16ms/step
Epoch 11/50
3/3 - 0s - loss: 0.0903 - accuracy: 0.9886 - val_loss: 0.4264 - val_accuracy: 0.8889 - 36ms/epoch - 12ms/step
Epoch 12/50
3/3 - 0s - loss: 0.0627 - accuracy: 0.9886 - val_loss: 0.4655 - val_accuracy: 0.8889 - 36ms/epoch - 12ms/step
Epoch 13/50
3/3 - 0s - loss: 0.0427 - accuracy: 1.0000 - val_loss: 0.5312 - val_accuracy: 0.8889 - 36ms/epoch - 12ms/step
Epoch 1/50
3/3 - 1s - loss: 0.5861 - accuracy: 0.8539 - val_loss: 0.4456 - val_accuracy: 0.8636 - 1s/epoch - 398ms/step
Epoch 2/50
3/3 - 0s - loss: 0.3507 - accuracy: 0.9101 - val_loss: 0.3435 - val_accuracy: 0.8636 - 38ms/epoch - 13ms/step
Epoch 3/50
3/3 - 0s - loss: 0

Epoch 7/50
3/3 - 0s - loss: 0.2277 - accuracy: 0.9326 - val_loss: 0.4692 - val_accuracy: 0.9091 - 35ms/epoch - 12ms/step
Epoch 8/50
3/3 - 0s - loss: 0.2057 - accuracy: 0.9326 - val_loss: 0.4556 - val_accuracy: 0.8864 - 38ms/epoch - 13ms/step
Epoch 9/50
3/3 - 0s - loss: 0.2010 - accuracy: 0.9551 - val_loss: 0.4243 - val_accuracy: 0.8864 - 36ms/epoch - 12ms/step
Epoch 10/50
3/3 - 0s - loss: 0.1549 - accuracy: 0.9551 - val_loss: 0.4071 - val_accuracy: 0.9091 - 35ms/epoch - 12ms/step
Epoch 11/50
3/3 - 0s - loss: 0.1517 - accuracy: 0.9663 - val_loss: 0.3940 - val_accuracy: 0.9091 - 37ms/epoch - 12ms/step
Epoch 12/50
3/3 - 0s - loss: 0.1315 - accuracy: 0.9663 - val_loss: 0.3948 - val_accuracy: 0.8864 - 36ms/epoch - 12ms/step
Epoch 13/50
3/3 - 0s - loss: 0.1447 - accuracy: 0.9663 - val_loss: 0.3919 - val_accuracy: 0.9091 - 37ms/epoch - 12ms/step
Epoch 1/50
3/3 - 1s - loss: 0.6694 - accuracy: 0.6517 - val_loss: 0.4418 - val_accuracy: 0.9091 - 1s/epoch - 351ms/step
Epoch 2/50
3/3 - 0s - loss: 0

Epoch 14/50
3/3 - 0s - loss: 0.0181 - accuracy: 0.9888 - val_loss: 0.2055 - val_accuracy: 0.9318 - 34ms/epoch - 11ms/step
Epoch 15/50
3/3 - 0s - loss: 0.0080 - accuracy: 1.0000 - val_loss: 0.1998 - val_accuracy: 0.9318 - 36ms/epoch - 12ms/step
Epoch 16/50
3/3 - 0s - loss: 0.0059 - accuracy: 1.0000 - val_loss: 0.2314 - val_accuracy: 0.9318 - 35ms/epoch - 12ms/step
Epoch 17/50
3/3 - 0s - loss: 0.0083 - accuracy: 1.0000 - val_loss: 1.0081 - val_accuracy: 0.8636 - 41ms/epoch - 14ms/step

Units: 256 Activation: relu Kernel size: 3 Learning rate: 0.001 Optimizer: RMSprop Dropout: 0.2
Epoch 1/50
3/3 - 1s - loss: 0.4924 - accuracy: 0.7841 - val_loss: 0.3074 - val_accuracy: 0.9111 - 812ms/epoch - 271ms/step
Epoch 2/50
3/3 - 0s - loss: 0.2780 - accuracy: 0.8977 - val_loss: 0.3222 - val_accuracy: 0.9111 - 36ms/epoch - 12ms/step
Epoch 3/50
3/3 - 0s - loss: 0.2386 - accuracy: 0.9091 - val_loss: 0.3195 - val_accuracy: 0.9111 - 36ms/epoch - 12ms/step
Epoch 4/50
3/3 - 0s - loss: 0.2013 - accuracy: 0.9

Epoch 6/50
3/3 - 0s - loss: 0.1982 - accuracy: 0.9432 - val_loss: 0.3735 - val_accuracy: 0.9111 - 35ms/epoch - 12ms/step
Epoch 7/50
3/3 - 0s - loss: 0.1465 - accuracy: 0.9432 - val_loss: 0.3916 - val_accuracy: 0.9111 - 34ms/epoch - 11ms/step
Epoch 8/50
3/3 - 0s - loss: 0.1449 - accuracy: 0.9432 - val_loss: 0.3879 - val_accuracy: 0.9111 - 35ms/epoch - 12ms/step
Epoch 9/50
3/3 - 0s - loss: 0.1215 - accuracy: 0.9545 - val_loss: 0.3868 - val_accuracy: 0.9111 - 37ms/epoch - 12ms/step
Epoch 10/50
3/3 - 0s - loss: 0.1371 - accuracy: 0.9545 - val_loss: 0.4031 - val_accuracy: 0.9111 - 35ms/epoch - 12ms/step
Epoch 11/50
3/3 - 0s - loss: 0.0731 - accuracy: 0.9773 - val_loss: 0.4507 - val_accuracy: 0.9111 - 36ms/epoch - 12ms/step
Epoch 12/50
3/3 - 0s - loss: 0.0534 - accuracy: 1.0000 - val_loss: 0.5061 - val_accuracy: 0.9111 - 36ms/epoch - 12ms/step
Epoch 1/50
3/3 - 1s - loss: 0.6040 - accuracy: 0.7528 - val_loss: 0.3643 - val_accuracy: 0.8636 - 816ms/epoch - 272ms/step
Epoch 2/50
3/3 - 0s - loss:

...

Epoch 47/50
3/3 - 0s - loss: 0.2952 - accuracy: 0.8764 - val_loss: 0.1160 - val_accuracy: 0.9773 - 34ms/epoch - 11ms/step
Epoch 48/50
3/3 - 0s - loss: 0.3205 - accuracy: 0.8989 - val_loss: 0.1412 - val_accuracy: 0.9318 - 35ms/epoch - 12ms/step
Epoch 49/50
3/3 - 0s - loss: 0.3608 - accuracy: 0.8764 - val_loss: 0.1562 - val_accuracy: 0.9091 - 35ms/epoch - 12ms/step
Epoch 50/50
3/3 - 0s - loss: 0.3130 - accuracy: 0.8652 - val_loss: 0.1271 - val_accuracy: 0.9318 - 35ms/epoch - 12ms/step

Units: 256 Activation: sigmoid Kernel size: 5 Learning rate: 1e-05 Optimizer: Nadam Dropout: 0.1
Epoch 1/50
3/3 - 2s - loss: 0.6771 - accuracy: 0.5909 - val_loss: 0.6883 - val_accuracy: 0.5778 - 2s/epoch - 548ms/step
Epoch 2/50
3/3 - 0s - loss: 0.6640 - accuracy: 0.6591 - val_loss: 0.6790 - val_accuracy: 0.5778 - 38ms/epoch - 13ms/step
Epoch 3/50
3/3 - 0s - loss: 0.6579 - accuracy: 0.6705 - val_loss: 0.6738 - val_accuracy: 0.5778 - 38ms/epoch - 13ms/step
Epoch 4/50
3/3 - 0s - loss: 0.6527 - accuracy: 0.659

Epoch 12/50
3/3 - 0s - loss: 0.4957 - accuracy: 0.8315 - val_loss: 0.3779 - val_accuracy: 0.9773 - 38ms/epoch - 13ms/step
Epoch 13/50
3/3 - 0s - loss: 0.4768 - accuracy: 0.8090 - val_loss: 0.3266 - val_accuracy: 0.9091 - 37ms/epoch - 12ms/step
Epoch 14/50
3/3 - 0s - loss: 0.4162 - accuracy: 0.8652 - val_loss: 0.2352 - val_accuracy: 0.9318 - 37ms/epoch - 12ms/step
Epoch 15/50
3/3 - 0s - loss: 0.3545 - accuracy: 0.9101 - val_loss: 0.1874 - val_accuracy: 0.9318 - 39ms/epoch - 13ms/step
Epoch 16/50
3/3 - 0s - loss: 0.3546 - accuracy: 0.8427 - val_loss: 0.1741 - val_accuracy: 0.9318 - 37ms/epoch - 12ms/step
Epoch 17/50
3/3 - 0s - loss: 0.3151 - accuracy: 0.8764 - val_loss: 0.1578 - val_accuracy: 0.9318 - 37ms/epoch - 12ms/step
Epoch 18/50
3/3 - 0s - loss: 0.3099 - accuracy: 0.8989 - val_loss: 0.1597 - val_accuracy: 0.9091 - 36ms/epoch - 12ms/step
Epoch 19/50
3/3 - 0s - loss: 0.3055 - accuracy: 0.8876 - val_loss: 0.1735 - val_accuracy: 0.9091 - 37ms/epoch - 12ms/step
Epoch 20/50
3/3 - 0s - l

Epoch 20/50
3/3 - 0s - loss: 0.2245 - accuracy: 0.9213 - val_loss: 0.3432 - val_accuracy: 0.8864 - 41ms/epoch - 14ms/step
Epoch 21/50
3/3 - 0s - loss: 0.2127 - accuracy: 0.9326 - val_loss: 0.3485 - val_accuracy: 0.8864 - 40ms/epoch - 13ms/step
Epoch 22/50
3/3 - 0s - loss: 0.2167 - accuracy: 0.9213 - val_loss: 0.3605 - val_accuracy: 0.8864 - 38ms/epoch - 13ms/step
Epoch 23/50
3/3 - 0s - loss: 0.2093 - accuracy: 0.9213 - val_loss: 0.3549 - val_accuracy: 0.8864 - 37ms/epoch - 12ms/step
Epoch 24/50
3/3 - 0s - loss: 0.2156 - accuracy: 0.9438 - val_loss: 0.3434 - val_accuracy: 0.8864 - 40ms/epoch - 13ms/step
Epoch 25/50
3/3 - 0s - loss: 0.2339 - accuracy: 0.9213 - val_loss: 0.3408 - val_accuracy: 0.8864 - 39ms/epoch - 13ms/step
Epoch 26/50
3/3 - 0s - loss: 0.1909 - accuracy: 0.9438 - val_loss: 0.3396 - val_accuracy: 0.8864 - 38ms/epoch - 13ms/step
Epoch 27/50
3/3 - 0s - loss: 0.1910 - accuracy: 0.9551 - val_loss: 0.3451 - val_accuracy: 0.9091 - 38ms/epoch - 13ms/step
Epoch 28/50
3/3 - 0s - l

Epoch 6/50
3/3 - 0s - loss: 0.6945 - accuracy: 0.6591 - val_loss: 0.6744 - val_accuracy: 0.5778 - 44ms/epoch - 15ms/step
Epoch 7/50
3/3 - 0s - loss: 0.6298 - accuracy: 0.6364 - val_loss: 0.6850 - val_accuracy: 0.5778 - 40ms/epoch - 13ms/step
Epoch 8/50
3/3 - 0s - loss: 0.6284 - accuracy: 0.6705 - val_loss: 0.6824 - val_accuracy: 0.5778 - 37ms/epoch - 12ms/step
Epoch 9/50
3/3 - 0s - loss: 0.6476 - accuracy: 0.6591 - val_loss: 0.6692 - val_accuracy: 0.5778 - 38ms/epoch - 13ms/step
Epoch 10/50
3/3 - 0s - loss: 0.6285 - accuracy: 0.6705 - val_loss: 0.6560 - val_accuracy: 0.5778 - 43ms/epoch - 14ms/step
Epoch 11/50
3/3 - 0s - loss: 0.6173 - accuracy: 0.6932 - val_loss: 0.6382 - val_accuracy: 0.5778 - 43ms/epoch - 14ms/step
Epoch 12/50
3/3 - 0s - loss: 0.5942 - accuracy: 0.6932 - val_loss: 0.6244 - val_accuracy: 0.5778 - 45ms/epoch - 15ms/step
Epoch 13/50
3/3 - 0s - loss: 0.5650 - accuracy: 0.7500 - val_loss: 0.6089 - val_accuracy: 0.5778 - 44ms/epoch - 15ms/step
Epoch 14/50
3/3 - 0s - loss:

Epoch 14/50
3/3 - 0s - loss: 0.5628 - accuracy: 0.7191 - val_loss: 0.4072 - val_accuracy: 0.9773 - 44ms/epoch - 15ms/step
Epoch 15/50
3/3 - 0s - loss: 0.5188 - accuracy: 0.7865 - val_loss: 0.3246 - val_accuracy: 0.9773 - 39ms/epoch - 13ms/step
Epoch 16/50
3/3 - 0s - loss: 0.4101 - accuracy: 0.8539 - val_loss: 0.2774 - val_accuracy: 0.9773 - 43ms/epoch - 14ms/step
Epoch 17/50
3/3 - 0s - loss: 0.4112 - accuracy: 0.8315 - val_loss: 0.2243 - val_accuracy: 0.9545 - 44ms/epoch - 15ms/step
Epoch 18/50
3/3 - 0s - loss: 0.3664 - accuracy: 0.8764 - val_loss: 0.1888 - val_accuracy: 0.9318 - 51ms/epoch - 17ms/step
Epoch 19/50
3/3 - 0s - loss: 0.3451 - accuracy: 0.8989 - val_loss: 0.1592 - val_accuracy: 0.9318 - 57ms/epoch - 19ms/step
Epoch 20/50
3/3 - 0s - loss: 0.3513 - accuracy: 0.8876 - val_loss: 0.1518 - val_accuracy: 0.9318 - 44ms/epoch - 15ms/step
Epoch 21/50
3/3 - 0s - loss: 0.3285 - accuracy: 0.9101 - val_loss: 0.1756 - val_accuracy: 0.9091 - 39ms/epoch - 13ms/step
Epoch 22/50
3/3 - 0s - l

In [20]:
print(best_acc)
print(baseline_values)

1.0
{'units': 256, 'activation': 'tanh', 'kernel_size': 3, 'learning_rate': 0.0001, 'optimizer': 'Adam', 'dropout': 0.1}


In [21]:
#STOP